# 🌊 Maritime Noise Analysis - DEMONSTRATION
## Gulf of Mexico | Proof of Concept

### ⚠️ IMPORTANT: This is a DEMONSTRATION
This notebook uses **NOAA AIS data from the Gulf of Mexico** to demonstrate:
- ✅ JOMOPANS-ECHO model implementation
- ✅ 250m grid methodology  
- ✅ Complete analysis pipeline
- ✅ Export capabilities (Folium, Mundi)

**The same methodology will be applied to Swedish waters** when HELCOM data is obtained.

### Why Gulf of Mexico?
- Available data NOW (NOAA public dataset)
- Validates model and pipeline
- Generates example visualizations
- Proves concept for hackathon

## Step 1: Load Available Data

In [11]:
# Initialize data loader
loader = AISDataLoader(data_path=data_raw)

# List available files
print("📁 Available NOAA AIS files:")
available_files = sorted(data_raw.glob("AIS_*.csv"))
for i, file in enumerate(available_files[:5], 1):
    print(f"   {i}. {file.name}")

if len(available_files) > 5:
    print(f"   ... and {len(available_files) - 5} more files")

📁 Data Loader initialized
   Path: /Users/raphaelanawa/Documents/GitHub/GIS_SymphonyLayer/data/raw/NOAA
   Required columns: MMSI, BaseDateTime, LAT, LON, SOG, VesselType, Length
📁 Available NOAA AIS files:


## Step 2: Load Sample Data

In [19]:
# Load first available file as demonstration
if available_files:
    sample_file = available_files[0]
    print(f"\n🔥 Loading demonstration file: {sample_file.name}\n")
    
    df_raw = loader.load_noaa_ais(filename=sample_file.name)
    
    if df_raw is not None:
        print(f"\n✅ Data loaded successfully!")
else:
    print("❌ No AIS data files found in the directory")
    print(f"   Expected location: {data_raw}")
    print("\n📝 Please ensure NOAA AIS CSV files are placed in this directory")

FileNotFoundError: [Errno 2] No such file or directory: 'GIS_SymphonyLayer/data/raw/NOAA/AIS_2024_01_01.csv'

## Step 3: Data Cleaning & Quality Control

In [ ]:
# Store raw data in loader
loader.df = df_raw

# Apply data quality filters (removes invalid data, keeps ALL vessel types)
df_clean = loader.clean_data()

# Explore cleaned data
loader.explore_data(detailed=False)

## Step 4: Geographic Focus Area

In [ ]:
# Define area of interest (Gulf of Mexico example)
# For Swedish waters, these bounds will change to Baltic Sea coordinates

print("\n🎯 Selecting Geographic Focus Area...")
print("   (For demo: Gulf of Mexico)")
print("   (For production: Swedish/Baltic Sea waters)\n")

# Gulf of Mexico sample area (adjust based on your data)
DEMO_BOUNDS = {
    'min_lon': -95.0,
    'max_lon': -88.0,
    'min_lat': 27.0,
    'max_lat': 30.0
}

# Apply geographic filter
df_filtered = loader.filter_geographic_area(
    min_lon=DEMO_BOUNDS['min_lon'],
    max_lon=DEMO_BOUNDS['max_lon'],
    min_lat=DEMO_BOUNDS['min_lat'],
    max_lat=DEMO_BOUNDS['max_lat']
)

print(f"\n📊 Filtered Data Summary:")
print(f"   Records: {len(df_filtered):,}")
print(f"   Unique vessels: {df_filtered['MMSI'].nunique():,}")
print(f"   Geographic extent: {DEMO_BOUNDS}")

## Step 5: Vessel Classification

In [ ]:
print("\n🚢 Classifying vessels using JOMOPANS-ECHO model...\n")

# Apply vessel classification
df_filtered['vessel_class'] = df_filtered.apply(
    lambda row: classify_vessel_from_ais(
        ais_type=row['VesselType'],
        speed_kn=row['SOG'],
        length_m=row['Length']
    ),
    axis=1
)

# Show classification results
print("📊 Vessel Classification Summary:")
print("="*60)
class_counts = df_filtered['vessel_class'].value_counts()
for vessel_type, count in class_counts.items():
    pct = count / len(df_filtered) * 100
    print(f"   {vessel_type:<20} {count:>8,} ({pct:>5.1f}%)")

print(f"\n   {'Total':<20} {len(df_filtered):>8,} (100.0%)")

## Step 6: Calculate Noise Emissions

In [ ]:
print("\n🔊 Calculating 125 Hz noise emissions for each vessel position...\n")

# Calculate noise emission for each AIS record
df_filtered['emission_125hz_dB'] = df_filtered.apply(
    lambda row: calculate_125hz_emission(
        vessel_type=row['vessel_class'],
        speed_kn=row['SOG'],
        length_m=row['Length']
    ),
    axis=1
)

print("✅ Noise emissions calculated!")
print("\n📊 Emission Statistics:")
print("="*60)
print(f"   Mean:     {df_filtered['emission_125hz_dB'].mean():.1f} dB re 1 µPa @ 1m")
print(f"   Median:   {df_filtered['emission_125hz_dB'].median():.1f} dB")
print(f"   Std Dev:  {df_filtered['emission_125hz_dB'].std():.1f} dB")
print(f"   Min:      {df_filtered['emission_125hz_dB'].min():.1f} dB")
print(f"   Max:      {df_filtered['emission_125hz_dB'].max():.1f} dB")

# Show emission by vessel class
print("\n📈 Average Emission by Vessel Class:")
print("="*60)
emission_by_class = df_filtered.groupby('vessel_class')['emission_125hz_dB'].agg(['mean', 'count']).sort_values('mean', ascending=False)
for vessel_class, row in emission_by_class.iterrows():
    print(f"   {vessel_class:<20} {row['mean']:>6.1f} dB  (n={int(row['count']):,})")

## Step 7: Create Spatial Grid (250m × 250m)

In [ ]:
print("\n🗺️ Creating 250m × 250m spatial grid...\n")

# Create grid generator
grid_generator = GridGenerator(
    bounds=(
        DEMO_BOUNDS['min_lon'],
        DEMO_BOUNDS['min_lat'],
        DEMO_BOUNDS['max_lon'],
        DEMO_BOUNDS['max_lat']
    ),
    cell_size_km=0.25  # 250 meters
)

# Generate grid
grid = grid_generator.create_grid()

print(f"\n✅ Grid created successfully!")
print(f"   Total cells: {len(grid):,}")
print(f"   Cell size: 250m × 250m")

## Step 8: Spatial Analysis - Assign to Grid

In [ ]:
print("\n📍 Assigning vessel positions to grid cells...\n")

# Initialize spatial analyzer
spatial_analyzer = SpatialNoiseAnalyzer(grid_generator)

# Assign positions to grid cells
gdf_with_cells = spatial_analyzer.assign_positions_to_grid(df_filtered)

print(f"\n✅ Spatial assignment complete!")

## Step 9: Calculate Grid-Level Metrics

In [ ]:
print("\n📊 Calculating noise metrics for each grid cell...\n")

# Calculate aggregated metrics per cell
grid_noise = spatial_analyzer.calculate_grid_noise_metrics(gdf_with_cells)

print(f"\n✅ Grid metrics calculated!")

# Show top noise hotspots
hotspots = spatial_analyzer.analyze_noise_hotspots(grid_noise, top_n=10)

## Step 10: Visualization - Static Plots

In [ ]:
print("\n📈 Creating static visualizations...\n")

# Create comprehensive visualization
fig = spatial_analyzer.plot_noise_grid(
    grid_noise, 
    title="Gulf of Mexico Maritime Noise Analysis - DEMONSTRATION"
)

plt.show()

print("\n✅ Static plots created!")

## Step 12: Export Complete Results

In [ ]:
print("\n🌐 Creating interactive Folium map...\n")

# Create output directory
viz_dir = data_outputs / 'demo_visualizations'
viz_dir.mkdir(exist_ok=True, parents=True)

# Export Folium map
folium_map = export_for_folium(
    grid_noise,
    output_path=viz_dir / 'gulf_of_mexico_noise_demo.html',
    center=None  # Auto-calculate center
)

print(f"\n✅ Interactive map created!")
print(f"   📂 Location: {viz_dir / 'gulf_of_mexico_noise_demo.html'}")
print(f"\n   🌐 Open the HTML file in your browser to explore!")

## Summary Statistics

In [ ]:
# Display final summary
cells_with_data = grid_noise[grid_noise['vessel_count'] > 0]

print("\n" + "="*70)
print("📋 ANALYSIS SUMMARY")
print("="*70)

print(f"\n🗺️  GRID COVERAGE:")
print(f"   Total grid cells: {len(grid_noise):,}")
print(f"   Cells with data: {len(cells_with_data):,} ({len(cells_with_data)/len(grid_noise)*100:.1f}%)")
print(f"   Empty cells: {len(grid_noise) - len(cells_with_data):,}")

print(f"\n🚢 VESSEL ACTIVITY:")
print(f"   Total observations: {int(grid_noise['vessel_count'].sum()):,}")
print(f"   Unique vessels: {df_filtered['MMSI'].nunique():,}")
print(f"   Max vessels per cell: {int(grid_noise['vessel_count'].max())}")

print(f"\n🔊 NOISE LEVELS (125 Hz):")
print(f"   Mean: {cells_with_data['noise_mean'].mean():.1f} dB re 1 µPa @ 1m")
print(f"   Median: {cells_with_data['noise_median'].median():.1f} dB")
print(f"   Range: {cells_with_data['noise_min'].min():.1f} - {cells_with_data['noise_max'].max():.1f} dB")

print(f"\n🎯 TOP NOISE HOTSPOT:")
top_cell = cells_with_data.nlargest(1, 'noise_mean').iloc[0]
print(f"   Cell ID: {int(top_cell['cell_id'])}")
print(f"   Average noise: {top_cell['noise_mean']:.1f} dB")
print(f"   Vessel count: {int(top_cell['vessel_count'])}")
print(f"   Dominant type: {top_cell['dominant_vessel_type']}")

print("\n" + "="*70)
print("✅ DEMONSTRATION SUCCESSFUL!")
print("="*70)
print("\n📝 Next Steps:")
print("   1. Review interactive map in browser")
print("   2. Analyze exported GeoJSON/CSV data")
print("   3. Apply same methodology to HELCOM data for Swedish waters")
print("   4. Customize analysis parameters as needed")